In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
import pandas_gbq
from datetime import datetime, timedelta, date
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [77]:
query2=f"""
SELECT
A.*,
B.RoundNumber
FROM `perceptive-ivy-290216.f1_api.results_race` A
INNER JOIN
`perceptive-ivy-290216.f1_api.schedules_update` B
ON A.GP=B.EventName AND A.Year=B.Year
# WHERE
# A.Year=2024
# AND B.Year=2024
-- AND
--  A.GP='Qatar Grand Prix'
"""

track2=pandas_gbq.read_gbq(query2,project_id,dialect='standard')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2379: UserWarning:

A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.



In [78]:
query3=f"""
SELECT
A.*,
B.RoundNumber
FROM `perceptive-ivy-290216.f1_api.results_sprint` A
INNER JOIN
`perceptive-ivy-290216.f1_api.schedules_update` B
ON A.GP=B.EventName AND A.Year=B.Year
# WHERE
# A.Year=2024
-- AND
-- A.GP='Qatar Grand Prix'
"""
track3=pandas_gbq.read_gbq(query3,project_id,dialect='standard')

In [79]:
year=2024

In [80]:
track_race=track2[track2["Year"]==year]
track_sprint=track3[track3["Year"]==year]

In [93]:
points_table=track_race[["RoundNumber","GP","FullName","Points"]]
points_table["Points"]=points_table["Points"].astype(int)
points_table=points_table.sort_values(by=["RoundNumber","FullName"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1499/308727433.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [94]:
points_table.tail()

,RoundNumber,GP,FullName,Points
6,14,Belgian Grand Prix,Oscar Piastri,18
3,14,Belgian Grand Prix,Pierre Gasly,0
19,14,Belgian Grand Prix,Sergio Perez,7
12,14,Belgian Grand Prix,Valtteri Bottas,0
1,14,Belgian Grand Prix,Yuki Tsunoda,0


In [95]:
points_table_sprint=track_sprint[["RoundNumber","GP","FullName","Points"]]
points_table_sprint["Points"]=points_table_sprint["Points"].astype(int)
points_table_sprint=points_table_sprint.sort_values(by=["RoundNumber","FullName"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1499/2916563121.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [96]:
points_table=pd.concat([points_table,points_table_sprint])
# points_table[points_table["GP"]=="British Grand Prix"].sort_values("FullName").head()

In [97]:
races = points_table['GP'].drop_duplicates()
results = points_table.pivot_table(index='FullName', columns='RoundNumber', values='Points', aggfunc='sum')
results.fillna(0)

results['total_points'] = results.sum(axis=1)
results = results.sort_values(by='total_points', ascending=False)
results.drop(columns='total_points', inplace=True)

# Use race name, instead of round no., as column names
results.columns = races
results=results.fillna(0)

In [98]:
fig_pts = px.imshow(
    results,
    text_auto=True,
    aspect='auto',  # Automatically adjust the aspect ratio
    width=1200, height=700,
    # color_continuous_scale=[[0,    'rgb(198, 219, 239)'],  # Blue scale
    #                         [0.25, 'rgb(107, 174, 214)'],
    #                         [0.5,  'rgb(33,  113, 181)'],
    #                         [0.75, 'rgb(8,   81,  156)'],
    #                         [1,    'rgb(8,   48,  107)']],
    labels={'x': 'Race',
            'y': 'FullName',
            'color': 'Points'}       # Change hover texts
)
fig_pts.update_xaxes(title_text='')      # Remove axis titles
fig_pts.update_yaxes(title_text='')
fig_pts.update_yaxes(tickmode='linear')  # Show all ticks, i.e. driver names
fig_pts.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey',
                 showline=False,
                 tickson='boundaries')              # Show horizontal grid only
fig_pts.update_xaxes(showgrid=False, showline=False)    # And remove vertical grid
fig_pts.update_layout(plot_bgcolor='rgba(0,0,0,0)')     # White background
fig_pts.update_layout(coloraxis_showscale=False)        # Remove legend
fig_pts.update_layout(xaxis=dict(side='top'))           # x-axis on top
fig_pts.update_layout(margin=dict(l=0, r=0, b=0, t=0))  # Remove border margins
fig_pts.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=13.5)),
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    xaxis_title="<b>GP</b>",
    yaxis_title="<b>Driver</b>",
    title_x=0.5,
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="BLACK"
    ),
    title_font_family="PT Sans Narrow"
)
fig_pts.show()

In [99]:
fig_pts.write_html("Driver and GP Points Matrix for the {} F1 Season.html".format(year))

In [100]:
#Position

In [104]:
pos_table=track_race[["RoundNumber","GP","FullName","Position"]]
pos_table["Position"]=pos_table["Position"].astype(int)
pos_table=pos_table.sort_values(by=["RoundNumber","FullName"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1499/349862856.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [105]:
pos_table_sprint=track_sprint[["RoundNumber","GP","FullName","Position"]]
pos_table_sprint["Position"]=pos_table_sprint["Position"].astype(int)
pos_table_sprint=pos_table_sprint.sort_values(by=["RoundNumber","FullName"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1499/3771742785.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [106]:
races_pos = pos_table['GP'].drop_duplicates()
results_pos = pos_table.pivot_table(index='FullName', columns='RoundNumber', values='Position', aggfunc='sum')

# results['total_points'] = results.sum(axis=1)
# results = results.sort_values(by='total_points', ascending=False)
# results.drop(columns='total_points', inplace=True)

# Use race name, instead of round no., as column names
results_pos.columns = races_pos
results_pos.fillna(0)
results_pos

GP,Bahrain Grand Prix,Saudi Arabian Grand Prix,Australian Grand Prix,Japanese Grand Prix,Chinese Grand Prix,Miami Grand Prix,Emilia Romagna Grand Prix,Monaco Grand Prix,Canadian Grand Prix,Spanish Grand Prix,Austrian Grand Prix,British Grand Prix,Hungarian Grand Prix,Belgian Grand Prix
FullName,,,,,,,,,,,,,,
Alexander Albon,15.0,11.0,11.0,20.0,12.0,18.0,20.0,9.0,17.0,18.0,15.0,9.0,14.0,12.0
Carlos Sainz,3.0,NaN,1.0,3.0,5.0,5.0,5.0,3.0,16.0,6.0,3.0,5.0,6.0,6.0
Charles Leclerc,4.0,3.0,2.0,4.0,4.0,3.0,3.0,1.0,19.0,5.0,11.0,14.0,4.0,3.0
Daniel Ricciardo,13.0,16.0,12.0,19.0,18.0,15.0,13.0,12.0,8.0,15.0,9.0,13.0,12.0,10.0
Esteban Ocon,17.0,13.0,16.0,15.0,11.0,10.0,14.0,17.0,10.0,10.0,12.0,16.0,18.0,9.0
Fernando Alonso,9.0,5.0,8.0,6.0,7.0,9.0,19.0,11.0,6.0,12.0,18.0,8.0,11.0,8.0
George Russell,5.0,6.0,17.0,7.0,6.0,8.0,7.0,5.0,3.0,4.0,1.0,19.0,8.0,20.0
Guanyu Zhou,11.0,18.0,15.0,18.0,14.0,14.0,15.0,16.0,15.0,13.0,17.0,18.0,19.0,19.0
Kevin Magnussen,12.0,12.0,10.0,13.0,16.0,19.0,12.0,20.0,12.0,17.0,8.0,12.0,15.0,14.0


In [107]:
fig_pos = px.imshow(
    results_pos,
    text_auto=True,
    aspect='auto',  # Automatically adjust the aspect ratio
    width=1200, height=700,
    color_continuous_scale=[[0,    'rgb(198, 219, 239)'],  # Blue scale
                            [0.25, 'rgb(107, 174, 214)'],
                            [0.5,  'rgb(33,  113, 181)'],
                            [0.75, 'rgb(8,   81,  156)'],
                            [1,    'rgb(8,   48,  107)']],
    labels={'x': 'Race',
            'y': 'Driver',
            'color': 'Position'}       # Change hover texts
)
fig_pos.update_xaxes(title_text='')      # Remove axis titles
fig_pos.update_yaxes(title_text='')
fig_pos.update_yaxes(tickmode='linear')  # Show all ticks, i.e. driver names
fig_pos.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey',
                 showline=False,
                 tickson='boundaries')              # Show horizontal grid only
fig_pos.update_xaxes(showgrid=False, showline=False)    # And remove vertical grid
fig_pos.update_layout(plot_bgcolor='rgba(0,0,0,0)')     # White background
fig_pos.update_layout(coloraxis_showscale=False)        # Remove legend
fig_pos.update_layout(xaxis=dict(side='top'))           # x-axis on top
fig_pos.update_layout(margin=dict(l=0, r=0, b=0, t=0))  # Remove border margins
fig_pos.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=13.5)),
    xaxis_title="<b>GP</b>",
    yaxis_title="<b>Driver</b>",
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="BLACK"
    ),
    title_font_family="PT Sans Narrow"
)
fig_pos.show()

In [108]:
fig_pos.write_html("Driver and Final GP Position Matrix for the {} F1 Season.html".format(year))

In [17]:
races_pos_sprint = pos_table_sprint['GP'].drop_duplicates()
results_pos_sprint = pos_table_sprint.pivot_table(index='FullName', columns='RoundNumber', values='Position', aggfunc='sum')

# results['total_points'] = results.sum(axis=1)
# results = results.sort_values(by='total_points', ascending=False)
# results.drop(columns='total_points', inplace=True)

# Use race name, instead of round no., as column names
results_pos_sprint.columns = races_pos_sprint
results_pos_sprint.fillna(0)
results_pos_sprint

GP,Chinese Grand Prix,Miami Grand Prix,Austrian Grand Prix
FullName,,,
Alexander Albon,17,13,17
Carlos Sainz,5,5,5
Charles Leclerc,4,2,7
Daniel Ricciardo,11,4,14
Esteban Ocon,13,15,11
Fernando Alonso,20,17,15
George Russell,8,12,4
Guanyu Zhou,9,11,20
Kevin Magnussen,10,18,9


In [109]:
fig_pos_sprint = px.imshow(
    results_pos_sprint,
    text_auto=True,
    width=1200, height=700,
    aspect='auto',  # Automatically adjust the aspect ratio
    color_continuous_scale=[[0,    'rgb(198, 219, 239)'],  # Blue scale
                            [0.25, 'rgb(107, 174, 214)'],
                            [0.5,  'rgb(33,  113, 181)'],
                            [0.75, 'rgb(8,   81,  156)'],
                            [1,    'rgb(8,   48,  107)']],
    labels={'x': 'Race',
            'y': 'Driver',
            'color': 'Position'}       # Change hover texts
)
fig_pos_sprint.update_xaxes(title_text='')      # Remove axis titles
fig_pos_sprint.update_yaxes(title_text='')
fig_pos_sprint.update_yaxes(tickmode='linear')  # Show all ticks, i.e. driver names
fig_pos_sprint.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey',
                 showline=False,
                 tickson='boundaries')              # Show horizontal grid only
fig_pos_sprint.update_xaxes(showgrid=False, showline=False)    # And remove vertical grid
fig_pos_sprint.update_layout(plot_bgcolor='rgba(0,0,0,0)')     # White background
fig_pos_sprint.update_layout(coloraxis_showscale=False)        # Remove legend
fig_pos_sprint.update_layout(xaxis=dict(side='top'))           # x-axis on top
fig_pos_sprint.update_layout(margin=dict(l=0, r=0, b=0, t=0))  # Remove border margins
fig_pos_sprint.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=13.5)),
    xaxis_title="<b>GP</b>",
    yaxis_title="<b>Driver</b>",
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="BLACK"
    ),
    title_font_family="PT Sans Narrow"
)
fig_pos_sprint.show()

In [110]:
fig_pos_sprint.write_html("Driver and Final Sprint Position Matrix for the {} F1 Season.html".format(year))

In [19]:
# Grid Position

In [111]:
grid_pos_table=track_race[["RoundNumber","GP","FullName","Position"]]
grid_pos_table["Position"]=grid_pos_table["Position"].astype(int)
grid_pos_table=grid_pos_table.sort_values(by=["RoundNumber","FullName"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1499/98208613.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [112]:
grid_pos_table_sprint=track_sprint[["RoundNumber","GP","FullName","Position"]]
grid_pos_table_sprint["Position"]=grid_pos_table_sprint["Position"].astype(int)
grid_pos_table_sprint=grid_pos_table_sprint.sort_values(by=["RoundNumber","FullName"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1499/3257691899.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [113]:
races_grid_pos = grid_pos_table['GP'].drop_duplicates()
results_grid_pos = grid_pos_table.pivot_table(index='FullName', columns='RoundNumber', values='Position', aggfunc='sum')

# results['total_points'] = results.sum(axis=1)
# results = results.sort_values(by='total_points', ascending=False)
# results.drop(columns='total_points', inplace=True)

# Use race name, instead of round no., as column names
results_grid_pos.columns = races_grid_pos
results_grid_pos.fillna(0)
results_grid_pos

GP,Bahrain Grand Prix,Saudi Arabian Grand Prix,Australian Grand Prix,Japanese Grand Prix,Chinese Grand Prix,Miami Grand Prix,Emilia Romagna Grand Prix,Monaco Grand Prix,Canadian Grand Prix,Spanish Grand Prix,Austrian Grand Prix,British Grand Prix,Hungarian Grand Prix,Belgian Grand Prix
FullName,,,,,,,,,,,,,,
Alexander Albon,15.0,11.0,11.0,20.0,12.0,18.0,20.0,9.0,17.0,18.0,15.0,9.0,14.0,12.0
Carlos Sainz,3.0,NaN,1.0,3.0,5.0,5.0,5.0,3.0,16.0,6.0,3.0,5.0,6.0,6.0
Charles Leclerc,4.0,3.0,2.0,4.0,4.0,3.0,3.0,1.0,19.0,5.0,11.0,14.0,4.0,3.0
Daniel Ricciardo,13.0,16.0,12.0,19.0,18.0,15.0,13.0,12.0,8.0,15.0,9.0,13.0,12.0,10.0
Esteban Ocon,17.0,13.0,16.0,15.0,11.0,10.0,14.0,17.0,10.0,10.0,12.0,16.0,18.0,9.0
Fernando Alonso,9.0,5.0,8.0,6.0,7.0,9.0,19.0,11.0,6.0,12.0,18.0,8.0,11.0,8.0
George Russell,5.0,6.0,17.0,7.0,6.0,8.0,7.0,5.0,3.0,4.0,1.0,19.0,8.0,20.0
Guanyu Zhou,11.0,18.0,15.0,18.0,14.0,14.0,15.0,16.0,15.0,13.0,17.0,18.0,19.0,19.0
Kevin Magnussen,12.0,12.0,10.0,13.0,16.0,19.0,12.0,20.0,12.0,17.0,8.0,12.0,15.0,14.0


In [114]:
fig_grid_pos = px.imshow(
    results_grid_pos,
    text_auto=True,
    aspect='auto',  # Automatically adjust the aspect ratio
    width=1200, height=700,
    color_continuous_scale=[[0,    'rgb(198, 219, 239)'],  # Blue scale
                            [0.25, 'rgb(107, 174, 214)'],
                            [0.5,  'rgb(33,  113, 181)'],
                            [0.75, 'rgb(8,   81,  156)'],
                            [1,    'rgb(8,   48,  107)']],
    labels={'x': 'Race',
            'y': 'Driver',
            'color': 'Position'}       # Change hover texts
)
fig_grid_pos.update_xaxes(title_text='')      # Remove axis titles
fig_grid_pos.update_yaxes(title_text='')
fig_grid_pos.update_yaxes(tickmode='linear')  # Show all ticks, i.e. driver names
fig_grid_pos.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey',
                 showline=False,
                 tickson='boundaries')              # Show horizontal grid only
fig_grid_pos.update_xaxes(showgrid=False, showline=False)    # And remove vertical grid
fig_grid_pos.update_layout(plot_bgcolor='rgba(0,0,0,0)')     # White background
fig_grid_pos.update_layout(coloraxis_showscale=False)        # Remove legend
fig_grid_pos.update_layout(xaxis=dict(side='top'))           # x-axis on top
fig_grid_pos.update_layout(margin=dict(l=0, r=0, b=0, t=0))  # Remove border margins
fig_grid_pos.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=13.5)),
    xaxis_title="<b>GP</b>",
    yaxis_title="<b>Driver</b>",
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="BLACK"
    ),
    title_font_family="PT Sans Narrow"
)
fig_grid_pos.show()

In [117]:
fig_grid_pos.write_html("Driver and Race Grid Position Matrix for the {} F1 Season.html".format(year))

In [118]:
races_grid_pos_sprint = grid_pos_table_sprint['GP'].drop_duplicates()
results_grid_pos_sprint = grid_pos_table_sprint.pivot_table(index='FullName', columns='RoundNumber', values='Position', aggfunc='sum')

# results['total_points'] = results.sum(axis=1)
# results = results.sort_values(by='total_points', ascending=False)
# results.drop(columns='total_points', inplace=True)

# Use race name, instead of round no., as column names
results_grid_pos_sprint.columns = races_grid_pos_sprint
results_grid_pos_sprint.fillna(0)
results_grid_pos_sprint

GP,Chinese Grand Prix,Miami Grand Prix,Austrian Grand Prix
FullName,,,
Alexander Albon,17,13,17
Carlos Sainz,5,5,5
Charles Leclerc,4,2,7
Daniel Ricciardo,11,4,14
Esteban Ocon,13,15,11
Fernando Alonso,20,17,15
George Russell,8,12,4
Guanyu Zhou,9,11,20
Kevin Magnussen,10,18,9


In [120]:
fig_grid_sprint_pos = px.imshow(
    results_grid_pos_sprint,
    text_auto=True,
    aspect='auto',  # Automatically adjust the aspect ratio
    width=1200, height=700,
    color_continuous_scale=[[0,    'rgb(198, 219, 239)'],  # Blue scale
                            [0.25, 'rgb(107, 174, 214)'],
                            [0.5,  'rgb(33,  113, 181)'],
                            [0.75, 'rgb(8,   81,  156)'],
                            [1,    'rgb(8,   48,  107)']],
    labels={'x': 'Race',
            'y': 'Driver',
            'color': 'Position'}       # Change hover texts
)
fig_grid_sprint_pos.update_xaxes(title_text='')      # Remove axis titles
fig_grid_sprint_pos.update_yaxes(title_text='')
fig_grid_sprint_pos.update_yaxes(tickmode='linear')  # Show all ticks, i.e. driver names
fig_grid_sprint_pos.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey',
                 showline=False,
                 tickson='boundaries')              # Show horizontal grid only
fig_grid_sprint_pos.update_xaxes(showgrid=False, showline=False)    # And remove vertical grid
fig_grid_sprint_pos.update_layout(plot_bgcolor='rgba(0,0,0,0)')     # White background
fig_grid_sprint_pos.update_layout(coloraxis_showscale=False)        # Remove legend
fig_grid_sprint_pos.update_layout(xaxis=dict(side='top'))           # x-axis on top
fig_grid_sprint_pos.update_layout(margin=dict(l=0, r=0, b=0, t=0))  # Remove border margins
fig_grid_sprint_pos.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=13.5)),
    xaxis_title="<b>GP</b>",
    yaxis_title="<b>Driver</b>",
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="BLACK"
    ),
    title_font_family="PT Sans Narrow"
)
fig_grid_sprint_pos.show()

In [121]:
fig_grid_sprint_pos.write_html("Driver and Sprint Race Grid Position Matrix for the {} F1 Season.html".format(year))